In [1]:
import sys
import os
from scipy.io import wavfile
import numpy as np
data_folder = "/Volumes/CB_RESEARCH/vocal_synthesis/"
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.init import *
from lasagne.nonlinearities import *

//anaconda/lib/python2.7/site-packages/Theano-0.8.0.dev0-py2.7.egg/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
fs, data = wavfile.read(data_folder + os.path.sep + "data.wav")

In [11]:
length_in_sec = float(len(data)) / fs
print "Length of wav file: %f hrs" % ((length_in_sec / 60) / 60)

Length of wav file: 3.024818 hrs


In [5]:
?wavfile.read

Do ZMUV normalisation on the data. We will need to keep the mean and standard deviation to backtransform the "predictions" later on.

In [3]:
data = np.asarray(data, dtype="float32")
mean = np.mean(data)
std = np.std(data)
data = (data - mean) / std

In [4]:
def generate_non_contig(chunk_size):
    X_train = []
    y_train = []
    b = 0
    while True:
        X_train.append( data[b*chunk_size : (b+1)*chunk_size] )
        y_train.append( data[(b+1)*chunk_size : (b+2)*chunk_size] )
        b += 1
        if (b+2)*chunk_size >= len(data):
            break
    return X_train, y_train

In [26]:
def generate_default(chunk_size):
    X_train = []
    y_train = []
    b = 0
    while True:
        this_chunk = data[b*chunk_size : (b+1)*chunk_size]
        if len(this_chunk) != chunk_size:
            break
        X_train.append( this_chunk[0 : len(this_chunk)-1] )
        y_train.append( [ this_chunk[len(this_chunk)-1] ] )
        b += 1
        if (b)*chunk_size >= len(data):
            break
    return np.asarray(X_train, dtype="float32"), np.asarray(y_train, dtype="float32")

In [27]:
X_train, y_train = generate_default(100 + 1)

In [28]:
X_train.shape, y_train.shape

((1725044, 100), (1725044, 1))

In [29]:
y_train

array([[ -2.21997925e-05],
       [ -2.21997925e-05],
       [ -2.21997925e-05],
       ..., 
       [ -2.21997925e-05],
       [ -2.21997925e-05],
       [ -2.21997925e-05]], dtype=float32)

In [30]:
chunk_size = 100
l_in = InputLayer( (None, chunk_size) )
l_hidden = DenseLayer(l_in, num_units=chunk_size*0.75, nonlinearity=leaky_rectify)
l_hidden2 = DenseLayer(l_hidden, num_units=chunk_size*0.5, nonlinearity=leaky_rectify)
l_hidden3 = DenseLayer(l_hidden2, num_units=chunk_size*0.75, nonlinearity=leaky_rectify)
l_out = DenseLayer(l_hidden3, num_units=1, nonlinearity=linear)

In [31]:
X = T.fmatrix('X')
y = T.fmatrix('y')
pred = lasagne.layers.get_output(l_out, X)
get_out = theano.function([X], pred)

In [32]:
params = lasagne.layers.get_all_params(l_out)

In [33]:
loss = lasagne.objectives.squared_error(pred, y)
loss = loss.mean()

In [34]:
learning_rate = 0.01
momentum = 0.9
updates = lasagne.updates.nesterov_momentum(
    loss, params, learning_rate=learning_rate, momentum=momentum)

In [35]:
train_fn = theano.function([X, y], loss, updates=updates)

In [ ]:
num_epochs = 100
for epoch in range(0, num_epochs):
    b = 0
    batch_size = 128
    train_losses = []
    while True:
        X_train_batch = X_train[b*batch_size : (b+1)*batch_size]
        y_train_batch = y_train[b*batch_size : (b+1)*batch_size]
        train_losses.append( train_fn(X_train_batch, y_train_batch) )
        b += 1
        if b*batch_size >= len(X_train):
            break
    print "train loss for epoch %i = %f" % (epoch, np.mean(train_losses))

train loss for epoch 0 = 0.009281
train loss for epoch 1 = 0.003250
train loss for epoch 2 = 0.002584
train loss for epoch 3 = 0.002304

In [42]:
np.mean(train_losses)

0.47639080989505334

In [22]:
wavfile.write(data=data[0:500000],rate=fs,filename="/tmp/file.wav")

In [ ]:
wavfile.